# 0. Setup Paths and Other Stuff

In [ ]:
# Add google drive to colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip list

In [ ]:
import os
import shutil

CUSTOM_MODEL_NAME = 'my_ssd_mobilenet'
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

paths = {
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),

    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),

    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),

    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME),
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'),
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'tfliteexport')
 }

files = {
    'PIPELINE_CONFIG':os.path.join(paths['MODEL_PATH'],CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], 'generate_tfrecord.py'),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], 'label_map.pbtxt'),
    'LABELMAP2': os.path.join(paths['ANNOTATION_PATH'], 'label_map.txt'),
    'TRAINING_SCRIPT': os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')
}

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)

# 1. Download TF Pretrained Models from TensorFlow Model Zoo and Install TFOD

In [ ]:
!pip install wget

In [ ]:
import wget

# Download pre-trained model
if not os.path.exists(os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME +'.tar.gz')):
    wget.download(PRETRAINED_MODEL_URL, f"{paths['PRETRAINED_MODEL_PATH']}")
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -xzvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

In [ ]:
# Clone the tensorflow models repository
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone --depth 1 https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
# Install Tensorflow Object Detection API
if os.name=='posix':
    # !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .

if os.name=='nt':
    # Install protobuf
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py && python setup.py build && python setup.py install
    # Install slim
    !cd Tensorflow/models/research/slim && pip install -e .

In [ ]:
# Test Installation
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
!python {VERIFICATION_SCRIPT}

In [ ]:
# Check if import works (might have to reload kernel)
import object_detection

# 2. Create Label Map

Label maps correspond index numbers to category names, so that when our convolution network predicts `1`, we know that this corresponds to `licence`.

In [ ]:
labels = [{'name':'licence', 'id':1}]

with open(files['LABELMAP'], 'w') as file:
    for label in labels:
        file.write("item { \n")
        file.write(f"\tname:\'{label['name']}\'\n")
        file.write(f"\tid:{label['id']}\n")
        file.write("}\n")
        
with open(files['LABELMAP2'], 'w') as file:
    for label in labels:
        file.write(f"{label['name']}\n")

# 3. Create TF records

In [ ]:
# OPTIONAL IF RUNNING ON COLAB

# Load train and test images from drive
ARCHIVE_PATH = os.path.join('drive', 'MyDrive', 'Colab Notebooks', 'archive.tar.gz')
if os.path.exists(ARCHIVE_PATH):
  !tar -xzvf "{ARCHIVE_PATH}" -C {paths["IMAGE_PATH"]}

# Load generate_tfrecord.py
if not os.path.exists(files['TF_RECORD_SCRIPT']):
  shutil.copy("/content/drive/MyDrive/Colab Notebooks/generate_tfrecord.py", paths['SCRIPTS_PATH'])

In [ ]:
 # Create tfrecords (my way)
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')}
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')}

# 4. Copy Pipeline Config to Training Folder and Update Config for Transfer Learning

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
# Copy Model Config to Training Folder
if not os.path.exists(files["PIPELINE_CONFIG"]):
    if os.name =='posix':
        !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['MODEL_PATH'], CUSTOM_MODEL_NAME)}
    if os.name == 'nt':
        !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['MODEL_PATH'], CUSTOM_MODEL_NAME)}

In [ ]:
# Load config
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as file:
    proto_str = file.read()
    text_format.Merge(proto_str, pipeline_config)

In [ ]:
# Set pipeline configuration
pipeline_config.model.ssd.num_classes = len(labels)

pipeline_config.train_config.batch_size = 8

# Do not touch ckpt-0!!!
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"

pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]

pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [ ]:
# Write config_text to pipeline.config
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as file:
    file.write(config_text)

# 5. Train and Evaluate the Model

In [ ]:
# OPTIONAL IF RUNNING ON COLAB

# Load model from drive
src = os.path.join("/content", "drive", "MyDrive", "Colab Notebooks", CUSTOM_MODEL_NAME)
dst = paths["CHECKPOINT_PATH"]
if os.path.exists(src):
  shutil.copytree(src, dst, dirs_exist_ok="true")

In [ ]:
!pip install tensorflow==2.10.0

In [ ]:
# Train model
# Note: num_train is the total steps added together from all iterations
command = f"python {files['TRAINING_SCRIPT']} --model_dir={paths['CHECKPOINT_PATH']} --pipeline_config_path={files['PIPELINE_CONFIG']} --num_train_steps=10000"
print(command)

In [ ]:
# Evaluate model
command = f"python {files['TRAINING_SCRIPT']} --model_dir={paths['CHECKPOINT_PATH']} --pipeline_config_path={files['PIPELINE_CONFIG']} --checkpoint_dir={paths['CHECKPOINT_PATH']}"
print(command)
# !{command}

# cd in eval folder
# tensorboard --logdir=. --port=8888

In [ ]:
# OPTIONAL IF RUNNING ON COLAB:
# Save model on drive

src = paths["CHECKPOINT_PATH"]
dst = os.path.join("/content", "drive", "MyDrive", "Colab Notebooks", CUSTOM_MODEL_NAME)
shutil.copytree(src, dst, dirs_exist_ok="true")

# 6. Freezing the Graph in Saved Model Format

In [ ]:
FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')

command = f"python {FREEZE_SCRIPT} --input_type=image_tensor --pipeline_config_path={files['PIPELINE_CONFIG']} --trained_checkpoint_dir={paths['CHECKPOINT_PATH']} --output_directory={paths['OUTPUT_PATH']}"
print(command)
!{command}

# 7. Conversion to TFLite (Does not work)

In [ ]:
TFLITE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'export_tflite_graph_tf2.py')

command = f"python {TFLITE_SCRIPT} --pipeline_config_path={files['PIPELINE_CONFIG']} --trained_checkpoint_dir={paths['CHECKPOINT_PATH']} --output_directory={paths['TFLITE_PATH']}"
print(command)
!{command}

In [ ]:
FROZEN_TFLITE_PATH = os.path.join(paths['TFLITE_PATH'], 'saved_model')
TFLITE_MODEL = os.path.join(paths['TFLITE_PATH'], 'saved_model', 'detect.tflite')

command = f"tflite_convert \
--saved_mod el_dir={FROZEN_TFLITE_PATH} \
--output_file={TFLITE_MODEL} \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=FLOAT \
--allow_custom_ops"

print(command)
!{command}

# 8. Zip and Export Models

In [ ]:
import shutil

# shutil.rmtree("Tensorflow/workspace/images/archive.tar.zip")
shutil.rmtree("/content/Tensorflow/workspace/models/my_ssd_mobilenet")
# shutil.copytree("/content/drive/MyDrive/Colab Notebooks/my_ssd_mobilenet", "/content/Tensorflow/workspace/models/my_ssd_mobilenet")

In [ ]:
# Compress training and testing images to archive.tar.gz
ARCHIVE_PATH = os.path.join(paths['IMAGE_PATH'], 'archive.tar.gz')

if not os.path.exists(ARCHIVE_PATH):
    !cd {os.path.join('Tensorflow', 'workspace', 'images')} && tar -czvf archive.tar.gz train test